# Bursar ZHB Closed after final reminder


## Workflow

Es gibt eine Eingabedatei von SLSP im Excel-Format. Vorgehen des Skripts:

- Excel-Datei einlesen und in einem Dataframe speichern

- Spalte 'Grund Abzuege': nur Zeilen berücksichtigen, bei denen der Wert "Closed after final reminder" steht (filtern).

- Spalte 'UserID' gruppieren und die Spalte 'Fakturierter Betrag' pro Benutzer summieren. 

- Summierten Betrag speichern in einer Variable 'Gesamtbetrag'.

- Pro UserID einen GET request auf die Alma user API abschicken und den user mittels der variable 'UserID' als primary_id im JSON-Format holen. Headers: 'Accept': 'application/json', 'Content-Type': 'application/json'. 

- user in einem JSON-Objekt speichern, Name und Vorname auslesen (Attribut 'first_name' und 'last_name')

- Bereich 'contact_info', 'address': Attribut "preferred": true berücksichtigen

- Daten der preferred address in einzelne Spalten speichern: 'line1', 'postal_code' 'city', 'country.value'

- Ist der Gesamtbetrag pro UserID 50.00 CHF oder höher: Aktion = "Rechnung". Ist der Gesamtbetrag kleiner als 50.00 CHF: Aktion = "Sperre"

- prüfen ob bereits ein user_block mit block_type.value 'CASH' vorhanden ist. Wenn ja: Spalte "Frühere Sperren": block_note und created_date dieses user_blocks speichern 

- Bei Aktion Sperre: String nach folgendem Muster als Variable 'Sperrnotiz' basteln: 

'Gesamtbetrag' CHF. Bitte an der Theke einkassieren und anschliessend Sperre entfernen, keine Ausleihe, bis Sperre gelöscht. Details: 'Fakturierter Betrag', 'Library code'/'Gebührentyp'/'Gebührendatum'", wiederhole dies für jeden Einzelbetrag dieser UserID.  

- Einen neuen (zusätzlichen) Block in folgender Struktur anlegen:

  "user_block": [
    {
      "block_type": {
        "value": "CASH",
        "desc": "Cash"
      },
      "block_description": {
        "value": "05",
        "desc": "Unpaid bill"
      },
      "block_status": "ACTIVE",
      "block_note": "'Sperrnotiz'",
      "created_by": "lit@zhbluzern.ch",
      "created_date": "(aktuellet Timestamp ISO)",
      "segment_type": "Internal"
    }


- Bei Aktion Rechnung: Benutzernotiz (user_note) anlegen "Rechnung 'gesamtbetrag' erstellt am..." anlegen

- Den angereicherten user mit einem PUT-request in Alma hochladen.



## Voraussetzungen für Python script:

Beachte bitte, dass du möglicherweise zusätzliche Bibliotheken installieren musst. Verwende 

        !pip install pandas requests openpyxl os datetime load_dotenv
        
in deiner Jupyter-Notebook-Zelle, um die benötigten Bibliotheken zu installieren.

Für die API-Keys wird ein .env file verwendet. 

Dateinamen anpassen: Variablen 'input_filename' und 'output_filename' entsprechend anpassen. 



In [ ]:
import pandas as pd
import requests
from datetime import datetime
from dotenv import load_dotenv
import os

# Api key
load_dotenv()
api_key = os.getenv('api_key_prod')
counter = 1
prod = 'False' # Durchlauf ohne Alma-Update

# 1. Lies die Eingabedatei 
input_filename = 'Bursar_15608_01.04.2024_test.xlsx'
df = pd.read_excel(input_filename)  

# 2. Filtere Zeilen, bei denen 'Grund Abzuege' gleich 'Closed after final reminder' ist
filtered_df = df[df['Grund Abzuege'] == 'Closed after final reminder']


# 3. Iteriere über die verbleibenden Zeilen
for user_id, user_group in filtered_df.groupby('UserID'):
    
    print("\n", counter)
    counter+= 1
    
    # 4. Hole den user mit der UserID als primary_id im JSON-Format    
    alma_api_url = f'https://api-eu.hosted.exlibrisgroup.com/almaws/v1/users/{user_id}?apikey={api_key}'
    headers = {'Accept': 'application/json', 'Content-Type': 'application/json'}
    response = requests.get(alma_api_url, headers=headers)

    # Überprüfe, ob die Anfrage erfolgreich war, bevor du versuchst, JSON zu decodieren
    if response.status_code == 200:
        print("Abfrage erfolgreich", user_id)
        user_data = response.json()

        # 5. Hole den vollen Namen und schreibe ihn in die Eingabedatei
        first_name = user_data.get('first_name', '')
        df.loc[df['UserID'] == user_id, 'Vorname'] = first_name
        last_name = user_data.get('last_name', '')
        df.loc[df['UserID'] == user_id, 'Nachname'] = last_name
        print(first_name, last_name)

        # 6. Hole die bevorzugte Adresse aus dem JSON-Objekt
        preferred_address = next((address for address in user_data.get('contact_info', {}).get('address', []) if address.get('preferred')), None)

        if preferred_address:
            address = preferred_address.get('line1', '')
            address2 = preferred_address.get('line2', '')
            postal_code = preferred_address.get('postal_code', '')
            city = preferred_address.get('city', '')
            country = preferred_address.get('country', {}).get('value', '')
            df.loc[df['UserID'] == user_id, 'Adresse'] = address
            df.loc[df['UserID'] == user_id, 'Adresse2'] = address2
            df.loc[df['UserID'] == user_id, 'PLZ'] = postal_code
            df.loc[df['UserID'] == user_id, 'Ort'] = city
            df.loc[df['UserID'] == user_id, 'Land'] = country
            print(address, address2, postal_code, city, country)
        else:
            print("Keine bevorzugte Adresse vorhanden")
            df.loc[df['UserID'] == user_id, 'Adresse'] = ''
            df.loc[df['UserID'] == user_id, 'Adresse2'] = ''
            df.loc[df['UserID'] == user_id, 'PLZ'] = ''
            df.loc[df['UserID'] == user_id, 'Ort'] = ''
            df.loc[df['UserID'] == user_id, 'Land'] = ''
            
        # 7. Überprüfe, ob bereits ein user_block mit block_type.value 'CASH' vorhanden ist
        user_blocks = user_data.get('user_block', [])
        cash_block = next((block for block in user_blocks if block.get('block_type', {}).get('value') == 'CASH'), None)
        user_block = next((block for block in user_blocks if block.get('block_type', {}).get('value') == 'USER'), None)

        if cash_block:
            # Schreibe Informationen zu vorherigem cash_block in die Eingabedatei
            sperrmeldung = f"Frühere CASH-Sperre vorhanden: {cash_block.get('block_note', '')}, "
            sperrmeldung += f"erstellt am {cash_block.get('created_date', '')}"
            print(sperrmeldung)
            df.loc[df['UserID'] == user_id, 'Frühere Sperren'] = sperrmeldung
            
        if user_block:
            # Schreibe Informationen zu vorherigem user_block in die Eingabedatei
            wrongaddress = f"Adress-Sperre vorhanden: {user_block.get('block_note', '')}, "
            wrongaddress += f"erstellt am {user_block.get('created_date', '')}"
            print(wrongaddress)
            df.loc[df['UserID'] == user_id, 'Adresssperre'] = wrongaddress
        
        # 8. Berechne den Gesamtbetrag und Aktion basierend auf dem Gesamtbetrag
        gesamtbetrag = user_group['Fakturierter Betrag'].sum()
        df.loc[df['UserID'] == user_id, 'Gesamtbetrag'] = gesamtbetrag
        print("Gesamtbetrag:", gesamtbetrag)

        if gesamtbetrag >= 50.00:
            aktion = 'Rechnung'
        else:
            aktion = 'Sperre'
        
        df.loc[df['UserID'] == user_id, 'Aktion'] = aktion
        print("Aktion:", aktion)


        # 9. Aktion Sperre: bilde den String für die Sperrnotiz
        if aktion == 'Sperre':
            sperrnotiz = f"Total {gesamtbetrag} CHF. Bitte an der Theke einkassieren und anschliessend Sperre entfernen, keine Ausleihe, bis Sperre gelöscht. Details: "
            sperrnotiz += ' / '.join([f"{betrag} CHF {library_code} {gebuehrentyp} {gebuehrendatum.strftime('%d-%m-%Y')}" for _, (betrag, library_code, gebuehrentyp, gebuehrendatum) in user_group[['Fakturierter Betrag', 'Library Code', 'Gebührentyp', 'Gebührendatum']].iterrows()])
            df.loc[df['UserID'] == user_id, 'Neue Notiz'] = sperrnotiz
            print(sperrnotiz)

            # Erstelle einen neuen user_block mit Sperrnotiz
            new_block = {
                "block_type": {"value": "CASH", "desc": "Cash"},
                "block_description": {"value": "05", "desc": "Unpaid bill"},
                "block_status": "ACTIVE",
                "block_note": sperrnotiz,
                "created_by": "lit@zhbluzern.ch",
                "created_date": datetime.now().isoformat(),
                "segment_type": "Internal"
            }

            user_blocks.append(new_block)
            user_data['user_block'] = user_blocks

        # 10. Aktion Rechnung: erstelle eine Nutzernotiz
        if aktion == 'Rechnung':
            rechnungsnotiz = f'ZHB-SAP: Rechnung über Gesamtbetrag von {gesamtbetrag} CHF erstellt am ' + datetime.now().strftime('%d-%m-%Y')
            new_note = {
              "note_type": {
                "value": "OTHER",
                "desc": "Other"
              },
              "note_text": rechnungsnotiz,
              "user_viewable": "false",
              "popup_note": "false",
              "created_by": "lit@zhbluzern.ch",
              "created_date": datetime.now().isoformat(),
              "segment_type": "Internal"
            }
            df.loc[df['UserID'] == user_id, 'Neue Notiz'] = rechnungsnotiz
            print(rechnungsnotiz)
            
            user_notes = user_data.get('user_note', [])
            user_notes.append(new_note)
            user_data['user_note'] = user_notes
        
        # 12: update user: lade den angereicherten Benutzer mit einem PUT-Request in Alma hoch
        
        if prod == 'True':
            put_response = requests.put(alma_api_url, json=user_data, headers=headers)
            if put_response.status_code == 200:
                print(f"Benutzer {user_id} aktualisiert. Statuscode: {put_response.status_code}")
            else:
                print("Benutzerupdate failed:", put_response.content)

    else:
        fehlermeldung = f"Fehler bei der API-Anfrage für Benutzer {user_id}. Statuscode: {response.status_code}"
        df.loc[df['UserID'] == user_id, 'Fehlermeldung'] = fehlermeldung
        print(fehlermeldung)

# 13. Speichere die aktualisierte Eingabedatei
output_filename = 'Bursar-09-04-2024-heka-Rechnungen-Sperren.xlsx'
df.to_excel(output_filename, index=False)
print(f'-------------------------------\nAktualisierte Eingabedatei wurde unter "{output_filename}" gespeichert.')
